<a href="https://colab.research.google.com/github/mssomie/Deep-Neural-Network/blob/main/Concrete_Strength_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building an Artificial Neural Network from Scratch


In [25]:
import random
import requests
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Define activation functions

Logistics Function

In [26]:
def logisitics_function (x):
  return 1/(1+np.exp(-x))


RELU Function


In [27]:
def RELU(x):
  return np.maximum(0,x)


Hyperbolic Function

In [28]:
def hyberbolic_tangent(x):
    function = (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
    derivative = 1 - function ** 2
    return function, derivative

In [29]:
print(logisitics_function(10))
print(RELU(10))
print(hyberbolic_tangent(10))

0.9999546021312976
10
(0.9999999958776926, 8.244614768671e-09)


# Define Fitness Function


In [30]:
def MSE(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

In [31]:
# Define fitness to be a measure of how good the model is
def fitness(y_true, y_pred):
    return 1/MSE(y_true, y_pred)

# Get Data

In [32]:
# Get dataset
url = "https://archive.ics.uci.edu/static/public/165/concrete+compressive+strength.zip"
response = requests.get(url)
with open("concrete_strength.zip", "wb") as file:
    file.write(response.content)

In [33]:
# Unzip dataset
!unzip concrete_strength.zip 

Archive:  concrete_strength.zip
replace Concrete_Data.xls? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [34]:
# Load data into dataframe
concrete_data = pd.read_excel(r"Concrete_Data.xls")

# View first five entries
concrete_data.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


In [35]:
# Check data information
print(concrete_data.shape)
concrete_data.info()

(1030, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Cement (component 1)(kg in a m^3 mixture)              1030 non-null   float64
 1   Blast Furnace Slag (component 2)(kg in a m^3 mixture)  1030 non-null   float64
 2   Fly Ash (component 3)(kg in a m^3 mixture)             1030 non-null   float64
 3   Water  (component 4)(kg in a m^3 mixture)              1030 non-null   float64
 4   Superplasticizer (component 5)(kg in a m^3 mixture)    1030 non-null   float64
 5   Coarse Aggregate  (component 6)(kg in a m^3 mixture)   1030 non-null   float64
 6   Fine Aggregate (component 7)(kg in a m^3 mixture)      1030 non-null   float64
 7   Age (day)                                              1030 non-null   int64  
 8   Concrete compressive strength(MPa, mega

In [36]:
# view summary statistics
concrete_data.describe()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.165631,73.895485,54.187136,181.566359,6.203112,972.918592,773.578883,45.662136,35.817836
std,104.507142,86.279104,63.996469,21.355567,5.973492,77.753818,80.175427,63.169912,16.705679
min,102.000000,0.000000,0.000000,121.750000,0.000000,801.000000,594.000000,1.000000,2.331808
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.707115
50%,272.900000,22.000000,0.000000,185.000000,6.350000,968.000000,779.510000,28.000000,34.442774
75%,350.000000,142.950000,118.270000,192.000000,10.160000,1029.400000,824.000000,56.000000,46.136287
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.599225


# Data Preprocessing

In [37]:
# Check for missing data
concrete_data.isnull().any().sum()

0

In [38]:
# Check for duplicates
print(concrete_data.duplicated().any())
print("Number of duplicate rows:", concrete_data.duplicated().sum())
duplicate_rows = concrete_data[concrete_data.duplicated()]
duplicate_rows

True
Number of duplicate rows: 25


,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
77,425.0,106.3,0.0,153.5,16.5,852.1,887.1,3,33.398217
80,425.0,106.3,0.0,153.5,16.5,852.1,887.1,3,33.398217
86,362.6,189.0,0.0,164.9,11.6,944.7,755.8,3,35.301171
88,362.6,189.0,0.0,164.9,11.6,944.7,755.8,3,35.301171
91,362.6,189.0,0.0,164.9,11.6,944.7,755.8,3,35.301171
100,425.0,106.3,0.0,153.5,16.5,852.1,887.1,7,49.201007
103,425.0,106.3,0.0,153.5,16.5,852.1,887.1,7,49.201007
109,362.6,189.0,0.0,164.9,11.6,944.7,755.8,7,55.895819
111,362.6,189.0,0.0,164.9,11.6,944.7,755.8,7,55.895819
123,425.0,106.3,0.0,153.5,16.5,852.1,887.1,28,60.294676


In [39]:
# Handle Duplicates
concrete_data.drop_duplicates(inplace = True)
print(concrete_data.duplicated().any())

False


**Split Data Into Training and Testing Sets**


In [40]:
x = concrete_data.drop(['Concrete compressive strength(MPa, megapascals) '], axis=1)
y = concrete_data['Concrete compressive strength(MPa, megapascals) ']
x_train, x_test, y_train, y_test= train_test_split(x,y, test_size=0.3, random_state=7)

In [41]:
# Standardise values
s_scaler = StandardScaler()
x_train_scaled= s_scaler.fit_transform(x_train)
x_test_scaled= s_scaler.fit_transform(x_test)

# Network


In [42]:
# TODO: fix for multiple hidden layers
class Multilayer_perceptron:

   # Initialize ANN
   def __init__(self, hidden_layers_neurons, activation_function, input_layer_neurons=8,  output_layer_neuron=1, bias_input =1):
       self.input_layer_neurons = input_layer_neurons
       self.hidden_layers_neurons = hidden_layers_neurons
       self.output_layer_neuron = output_layer_neuron
       self.weight = []
       self.bias = []

       self.activation_function = activation_function

       # Input layer to hidden layer
       self.weight.append(np.random.rand(hidden_layers_neurons[0],input_layer_neurons))
       self.bias.append(np.random.rand(hidden_layers_neurons[0],bias_input))

      # Hidden layer to hidden layer
       for i in range(1, len(hidden_layers_neurons)):
        self.weight.append(np.random.rand(hidden_layers_neurons[i], hidden_layers_neurons[i-1]))
        self.bias.append(np.random.rand(hidden_layers_neurons[i], bias_input))


       # Hidden layer to output layer
       self.weight.append(np.random.rand(output_layer_neuron,hidden_layers_neurons[-1]))
       self.bias.append(np.random.rand(output_layer_neuron,bias_input))

    # Set the weights and biases based on the PSO's particles
   def set_weights(self, particle):
        # Let us assume that the each particle contains a comnbination of weights and biases
        # We split the array evenly
        # TODO: We have to make sure the length (dimension) of each particle is the sum of the length of the weight and biases.
        split_point = len(self.weight)
        self.weight = particle[:split_point-1]
        self.bias = particle[split_point:]
        
    

   def forward_pass(self, input):
        # Make input a 2D array
        input = np.array(input)

       # using Z = Wx + b
        z_hidden=[]
        a_hidden=[]
        for i in range(len(self.weight)):
            if i == 0:
                z = np.dot(input, self.weight[i].T) + self.bias[i].T
            else:
                z = np.dot(a_hidden[i-1], self.weight[i].T) + self.bias[i].T
            z_hidden.append(z)

            if i < len(self.weight)-1:
                a_hidden.append(self.activation_function(z))
            else:
                a_hidden.append(z)
            


        return a_hidden[-1]
   
  

In [43]:
# Test the Multilayer_perceptron class
def test_multilayer_perceptron():
    # Define test parameters
    input_neurons = 8
    hidden_layers = [10, 8, 6]
    output_neurons = 1
    activation_func = RELU  # Using the RELU function defined earlier

    # Create an instance of Multilayer_perceptron
    mlp = Multilayer_perceptron(hidden_layers, activation_func, input_neurons, output_neurons)

    # Check if the network structure is correct
    assert len(mlp.weight) == len(hidden_layers) + 1, "Incorrect number of weight matrices"
    assert len(mlp.bias) == len(hidden_layers) + 1, "Incorrect number of bias vectors"

    # Check dimensions of weight matrices and bias vectors
    assert mlp.weight[0].shape == (hidden_layers[0], input_neurons), "Incorrect shape of first weight matrix"
    for i in range(1, len(hidden_layers)):
        assert mlp.weight[i].shape == (hidden_layers[i], hidden_layers[i-1]), f"Incorrect shape of weight matrix {i}"
    assert mlp.weight[-1].shape == (output_neurons, hidden_layers[-1]), "Incorrect shape of output weight matrix"

    for i, neurons in enumerate(hidden_layers):
        assert mlp.bias[i].shape == (neurons, 1), f"Incorrect shape of bias vector {i}"
    assert mlp.bias[-1].shape == (output_neurons, 1), "Incorrect shape of output bias vector"

    # Test forward pass
    input_data = np.random.rand(input_neurons, 1)  # Create random input data
    output = mlp.forward_pass(input_data)

    assert output.shape == (output_neurons, 1), "Incorrect output shape"

    print("All tests passed!")

# Run the test
test_multilayer_perceptron()


ValueError: shapes (8,1) and (8,10) not aligned: 1 (dim 1) != 8 (dim 0)

In [48]:
MLP = Multilayer_perceptron(hidden_layers_neurons= [64,32],activation_function=logisitics_function)
print(x_test_scaled.shape)
y_pred_trial= MLP.forward_pass(x_test_scaled)
print(y_pred_trial)
print(y_pred_trial.shape)
print(y_test.shape)
print(np.mean((y_test - y_pred_trial.flatten()) ** 2))
def assess_fitness(MLP,particle,x_train_scaled, y_test):
      
      MLP.set_weights(particle)
      y_pred = MLP.forward_pass(x_train_scaled)
      # using MSE
      fitness = np.mean((y_test - y_pred) ** 2)
      return fitness



(302, 8)
[[0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.9999999

**Particle Swarm Optimization**

In [50]:
class Particle_Swarm_Optimization:
    def __init__(self, swarm_size, dimensions, fitness_function, alpha, beta, gamma, delta, epsilon):
        self.swarm_size = swarm_size
        self.dimensions = dimensions
        self.fitness_function = fitness_function
        self.particles_fittest = []
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.delta = delta
        self.epsilon = epsilon
        self.velocities = []



    def initiailize_swarm(self):
        # Initialize the particles with random positions
        self.particles = np.random.rand(self.swarm_size, self.dimensions)
        #self.velocities = np.zeros((self.swarm_size, self.dimensions))

        # Initialize the personal best positions as the particles' initial positions
        self.personal_best_positions = self.particles.copy()
        self.global_best_position = self.particles[0].copy()
        self.informants_best_position = self.particles.copy()
        
        # Initialize particle velocity  using v = x(t) - x(t−1)
        for i in range (self.swarm_size):
            self.point_a= np.random.rand(self.swarm_size, self.dimensions)
            self.point_b= np.random.rand(self.swarm_size, self.dimensions)
            self.vector= self.point_b- self.point_a
            self.velocities.append(self.vector* 1/2)
        pass

    
    def update_fitness(self, particle):
        # Update the personal best position if the current particle has a higher fitness value
        for i, val_i in enumerate(particle):
            if fitness(val_i) > fitness(self.personal_best_positions[i]):
                self.personal_best_positions[i] = val_i
        # Update the global best position if the fitness of the personal best has a higher value
            if fitness(self.personal_best_positions[i]) > fitness(self.global_best_position[i]):
                self.global_best_position[i] = self.personal_best_positions[i]
        return  

    # Select informants for each particle and return the best informant
    def informants_best(self, particle_index, informants_count=5):
        # Get the indices of all particles except current particle and randomly choose informants_count-1 particles
        other_indices = np.array([i for i in range(self.swarm_size) if i != particle_index])
        chosen_indices = np.random.choice(other_indices, size =informants_count-1, replace=False)

        # Add the particle itself as one of the informants
        chosen_indices = np.append(chosen_indices, particle_index)

        # Get the index of the informant with the highest fitness
        informants_best = max(chosen_indices, key=lambda x: fitness(self.personal_best_positions[x]))
        return self.personal_best_positions[informants_best]



    def update_velocity(self):
        # using vi ← αvi+b(xi∗−xi)+c(xi+−xi)+d(xi!−xi)
        for i in range (self.swarm_size):
            initial_velocity = self.alpha * self.velocities[i] 
            personal_velocity = self.beta * (self.personal_best_positions[i] - self.particles[i])
            informants_velocity = self.gamma * (self.informants_best(i) - self.particles[i] )
            global_velocity = self.delta * (self.global_best_position[i] - self.particles[i])
            self.velocities[i] = initial_velocity + personal_velocity + informants_velocity + global_velocity


    def update_position(self ):
        # Using x ← x + ε v
        for i in range(self.swarm_size):
            self.particles[i] = self.particles[i] + (self.epsilon * self.velocities[i])
        

    def optimize(self, max_iterations):
        self.intiialize_swarm()
        for i in range(max_iterations):
            for j in range(self.swarm_size):
                self.assess_fitness(self.particles[j])
            self.update_velocity()
            self.update_position()

        pass
        
